In [ ]:
# default_exp parquet

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#hide
# stellt sicher, dass beim verändern der core library diese wieder neu geladen wird
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Parquet
Notebook to explore the basics of Parquet
Links
* Spark documentation https://spark.apache.org/docs/2.4.5/

In [ ]:
# common imports
import os
import zipfile
from bfh_cas_bgd_fs2020_sa.core import *

In [ ]:
# Check the current working dir
print(os.getcwd())

C:\ieu\projects\bfh_cas_bgd_fs2020_sa


In [ ]:
# Basic Definition
data_folder = "./data/" # folder with testdata
temp_folder = "./tmp/"
parquet_folder = "./parquet/"
data_files = ['2019q3.zip','2019q4.zip']

## Init Spark
> This code initialises the SparkSession and therefore the SparkContext. Pressing the link "Spark UI" opens the Spark UI for this session.

In [ ]:
# init Spark
spark = get_spark_session() # Session anlegen
spark # display the moste important information of the session

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:62782)
Traceback (most recent call last):
  File "C:\ieu\Anaconda3\envs\spark\lib\site-packages\py4j\java_gateway.py", line 1145, in send_command
    self.socket.sendall(command.encode("utf-8"))
ConnectionResetError: [WinError 10054] Eine vorhandene Verbindung wurde vom Remotehost geschlossen

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ieu\Anaconda3\envs\spark\lib\site-packages\py4j\java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "C:\ieu\Anaconda3\envs\spark\lib\site-packages\py4j\java_gateway.py", line 1149, in send_command
    "Error while sending", e, proto.ERROR_ON_SEND)
py4j.protocol.Py4JNetworkError: Error while sending

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ieu\Anaconda3\envs\spark\lib\si

Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:62782)

## Datafiles
> The directory contains two zipfiles (2019q3.zip, 2019q4.zip). Each of them contains 4 csv files. The columns and the relation between these files are described in the readme.htm.
> Each zip file contains all quarterly and yearly reports that were filled during the quarter denoted by the filename.

## Working with the data

### Unpacking
> In a first step, the content of the files are unzipped and placed in separated folders

In [ ]:
for data_file in data_files:
    path_to_zip_file = data_folder + data_file
    directory_to_extract_to = temp_folder + data_file[:-4]
    with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
        zip_ref.extractall(directory_to_extract_to)

> the sizes of the directories

In [ ]:
print('Data folder: ', get_size_format(get_directory_size(data_folder)))
print('Temp folder: ', get_size_format(get_directory_size(temp_folder)))

Data folder:  76.58MB
Temp folder:  748.04MB


### Using Parquet
> let us read a csv file and store it as a parquet file
> * API doc of the csv reader: https://spark.apache.org/docs/2.4.5/api/python/pyspark.sql.html?highlight=parquet#pyspark.sql.DataFrameReader.csv
> * API doc of the parquet writer: https://spark.apache.org/docs/2.4.5/api/python/pyspark.sql.html?highlight=parquet#pyspark.sql.DataFrameWriter.parquet

In [ ]:
# imports that are used in this section
from pyspark.sql.types import StructType, StructField, StringType, DateType, DoubleType, IntegerType

In [ ]:
# as a test csv file, "num.txt" from the folder 2019q3 is used
test_file = temp_folder + '2019q3/num.txt'
print('size of test file: ', get_size_format(os.path.getsize(test_file)))

size of test file:  236.70MB


#### Reading a CSV file
> As a first step, the csv file has to be loaded into a spark df. 
> The file has a header row and the columns are separated by a TAB (\t).

In [ ]:
df_test_num = spark.read.csv(test_file, sep='\t', header=True)

> When checking the format in the next cell, we see that all columns were read as a string. That is ok for most of the columns but when checking the definitions for the num.txt file in the readme.htm we see, that ddate is a date in the format 'yyyymmdd', qtrs and coreg are 'int' and value is a float.

In [ ]:
print('first row: ', df_test_num.head(1))
print('count     :', df_test_num.count())
df_test_num.printSchema()

first row:  [Row(adsh='0001625376-19-000017', tag='EntityPublicFloat', version='dei/2014', coreg=None, ddate=datetime.date(2018, 4, 30), qtrs=0, uom='USD', value=0.0, footnote=None)]
count     : 2325267
root
 |-- adsh: string (nullable = true)
 |-- tag: string (nullable = true)
 |-- version: string (nullable = true)
 |-- coreg: integer (nullable = true)
 |-- ddate: date (nullable = true)
 |-- qtrs: integer (nullable = true)
 |-- uom: string (nullable = true)
 |-- value: double (nullable = true)
 |-- footnote: string (nullable = true)



> Acutally, spark can try to infer the types of the columns from the data itself, so lets try that by using the "inferSchema" option

In [ ]:
df_test_num = spark.read.csv(test_file, sep='\t', header=True, inferSchema=True)

> As we can see in the next cell, we were only partially sucessfull. The reader was able to detect that qtrs is an integer and that value is a double. But it failed to recognize that ddate is actually a date and that coreg should be an int. That was to be expected: ddate looks like int and the coreg field is only used in special situations, so there is a good change that its field is None for all entries in the file. It looks as if we have to define the schema by hand

In [ ]:
print(df_test_num.head(1))
df_test_num.printSchema()

[Row(adsh='0001625376-19-000017', tag='EntityPublicFloat', version='dei/2014', coreg=None, ddate=20180430, qtrs=0, uom='USD', value=0.0, footnote=None)]
root
 |-- adsh: string (nullable = true)
 |-- tag: string (nullable = true)
 |-- version: string (nullable = true)
 |-- coreg: string (nullable = true)
 |-- ddate: integer (nullable = true)
 |-- qtrs: integer (nullable = true)
 |-- uom: string (nullable = true)
 |-- value: double (nullable = true)
 |-- footnote: string (nullable = true)



> All necessary classes to define a schema are located inside the package pyspark.sql.types and for our example we need the following import
from pyspark.sql.types import StructType, StructField, StringType, DateType, DoubleType, IntegerType

> An important point is that the dateFormat has to be defined as parameter when calling spark.read.csv() 

In [ ]:
schema = StructType([StructField("adsh",    StringType(),  True),\
                     StructField("tag",     StringType(),  True),\
                     StructField("version", StringType(),  True),\
                     StructField("coreg",   IntegerType(), True),\
                     StructField("ddate",   DateType(),    True),\
                     StructField("qtrs",    IntegerType(), True),\
                     StructField("uom",     StringType(),  True),\
                     StructField("value",   DoubleType(),  True),\
                     StructField("footnote",StringType(),  True)\
                    ])
df_test_num = spark.read.csv(test_file, sep='\t', header=True, dateFormat="yyyyMMdd", schema = schema)

In [ ]:
print(df_test_num.head(1))
df_test_num.printSchema()

[Row(adsh='0001625376-19-000017', tag='EntityPublicFloat', version='dei/2014', coreg=None, ddate=datetime.date(2018, 4, 30), qtrs=0, uom='USD', value=0.0, footnote=None)]
root
 |-- adsh: string (nullable = true)
 |-- tag: string (nullable = true)
 |-- version: string (nullable = true)
 |-- coreg: integer (nullable = true)
 |-- ddate: date (nullable = true)
 |-- qtrs: integer (nullable = true)
 |-- uom: string (nullable = true)
 |-- value: double (nullable = true)
 |-- footnote: string (nullable = true)



#### simple write as Parquet
> As first version the dataframe is stored directly in parquet format without additional options

In [ ]:
parquet_folder_pure = parquet_folder+"pure/"
df.write.parquet(parquet_folder_pure, mode="overwrite") # mode 'overwrite' overwrites the data, if they are already present

In [ ]:
print('size of parquet_folder_pure: ', get_size_format(get_directory_size(parquet_folder_pure)))

size of parquet_folder_pure:  22.78MB


> Parquet was able to compress the data down to 10% of the orginal size. 

#### using partitions
> Parquet can also store the data in different partitions wich will create a new directory for every partition.

> in a first test we will partition by the "adsh" column which is the identifier for the different reports

In [ ]:
parquet_folder_by_adsh = parquet_folder+"byadsh/"
df.write.partitionBy('adsh').parquet(parquet_folder_by_adsh, mode="overwrite")

Py4JJavaError: An error occurred while calling o250.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:198)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:159)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOpera